In [1]:
import os
import sys
sys.path.append("/Users/shashanks./Downloads/Installations/ddn/")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from ddn.pytorch.node import *
from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
from bernstein import bernstein_coeff_order10_new

torch.set_printoptions(8)

#### CUDA Initializations

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cpu device


#### Initializations

In [3]:
rho_obs = 0.3
rho_eq = 10.0
weight_smoothness = 10

In [4]:
num = 20
t_fin = 8.0
a_obs = 1.0
b_obs = 1.0

tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]

In [5]:
x_obs_temp = np.hstack((-2.0, -0.79, 3.0, 4.0))
y_obs_temp = np.hstack((-2.0, 1.0, -0.80, 2.0))
num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]

In [6]:
A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
A_obs = np.tile(P, (num_obs, 1))
Q_smoothness = np.dot(Pddot.T, Pddot)

In [7]:
maxiter = 300
eps = 10 ** (-8.0)
num_tot = num_obs * num

#### Tensor initializations

In [8]:
P_tensor = torch.tensor(P, dtype=torch.double)
Pddot_tensor = torch.tensor(Pddot, dtype=torch.double)
A_eq_tensor = torch.tensor(A_eq, dtype=torch.double)
A_obs_tensor = torch.tensor(A_obs, dtype=torch.double)
Q_smoothness_tensor = torch.tensor(Q_smoothness, dtype=torch.double)
x_obs_tensor = torch.tensor(x_obs, dtype=torch.double)
y_obs_tensor = torch.tensor(y_obs, dtype=torch.double)
P_tensor.size(), A_eq_tensor.size(), A_obs_tensor.size(), Q_smoothness_tensor.size(), x_obs_tensor.size(), y_obs_tensor.size()

(torch.Size([20, 11]),
 torch.Size([6, 11]),
 torch.Size([80, 11]),
 torch.Size([11, 11]),
 torch.Size([4, 20]),
 torch.Size([4, 20]))

In [ ]:
# sol = torch.randn(10, 182, dtype=torch.double)
# params = torch.randn(10, 12, dtype=torch.double)
# lamda_x = torch.randn(10, 11, dtype=torch.double)
# lamda_y = torch.randn(10, 11, dtype=torch.double)

# sol = sol.transpose(0, 1)
# params = params.transpose(0, 1)
# lamda_x = lamda_x.transpose(0, 1)
# lamda_y = lamda_y.transpose(0, 1)
# sol.size(), params.size(), lamda_x.size(), lamda_y.size()

In [ ]:
# params[:, 4]

#### Defining objectives

In [ ]:
bx_eq_tensor, by_eq_tensor = torch.split(params, 6, dim=0)

c_x = sol[0:nvar]
c_y = sol[nvar:2 * nvar]
alpha_obs = sol[2 * nvar: 2 * nvar + num_tot]
d_obs = sol[2 * nvar + num_tot:]

cost_smoothness_x = 0.5 * weight_smoothness * torch.diag(torch.matmul(c_x.T, torch.matmul(Q_smoothness_tensor, c_x)))
cost_smoothness_y = 0.5 * weight_smoothness * torch.diag(torch.matmul(c_y.T, torch.matmul(Q_smoothness_tensor, c_y)))

temp_x_obs = d_obs * torch.cos(alpha_obs) * a_obs
b_obs_x = x_obs_tensor.view(-1, 1) + temp_x_obs

temp_y_obs = d_obs * torch.sin(alpha_obs) * b_obs
b_obs_y = y_obs_tensor.view(-1, 1) + temp_y_obs

cost_obs_x = 0.5 * rho_obs * (torch.sum((torch.matmul(A_obs_tensor, c_x) - b_obs_x) ** 2, axis=0))
cost_obs_y = 0.5 * rho_obs * (torch.sum((torch.matmul(A_obs_tensor, c_y) - b_obs_y) ** 2, axis=0))
cost_slack = rho_obs * torch.sum(F.relu(1 - d_obs), axis=0)

cost_eq_x = 0.5 * rho_eq * torch.sum((torch.matmul(A_eq_tensor, c_x) - bx_eq_tensor) ** 2, axis=0)
cost_eq_y = 0.5 * rho_eq * torch.sum((torch.matmul(A_eq_tensor, c_y) - by_eq_tensor) ** 2, axis=0)

cost_x = cost_smoothness_x + cost_obs_x + cost_eq_x - torch.diag(torch.matmul(lamda_x.transpose(0, 1), c_x))
cost_y = cost_smoothness_y + cost_obs_y + cost_eq_y - torch.diag(torch.matmul(lamda_y.transpose(0, 1), c_y))
cost = cost_x + cost_y + eps * torch.sum(c_x ** 2, axis=0) + eps * torch.sum(c_y ** 2, axis=0) + eps * torch.sum(d_obs ** 2, axis=0) + eps * torch.sum(alpha_obs ** 2, axis=0) + cost_slack

In [ ]:
cost

#### Compute Solution

In [ ]:
def compute_solution(b_tensor, lamda_x, lamda_y):
    d_obs = torch.zeros(num_obs, num)
    alpha_obs = torch.zeros(num_obs, num)
    ones_tensor = torch.ones((num_obs, num), dtype=torch.double)
    bx_eq_tensor, by_eq_tensor = torch.split(b_tensor, 6, dim=0)
    cost_smoothness = weight_smoothness * torch.matmul(Pddot_tensor.T, Pddot_tensor)
    cost = cost_smoothness + rho_obs * torch.matmul(A_obs_tensor.T, A_obs_tensor) + rho_eq * torch.matmul(A_eq_tensor.T, A_eq_tensor)
    
    for i in range(maxiter):
        temp_x_obs = d_obs * torch.cos(alpha_obs) * a_obs
        temp_y_obs = d_obs * torch.sin(alpha_obs) * b_obs
        
        b_obs_x = x_obs_tensor.view(num * num_obs) + temp_x_obs.view(num * num_obs)
        b_obs_y = y_obs_tensor.view(num * num_obs) + temp_y_obs.view(num * num_obs)
        
        lincost_x = -lamda_x - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_x) - rho_eq * torch.matmul(A_eq_tensor.T, bx_eq_tensor)
        lincost_y = -lamda_y - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_y) - rho_eq * torch.matmul(A_eq_tensor.T, by_eq_tensor)

        lincost_x = lincost_x.view(-1, 1)
        lincost_y = lincost_y.view(-1, 1)
        
        sol_x, _ = torch.solve(lincost_x, -cost)
        sol_y, _ = torch.solve(lincost_y, -cost)

        sol_x = sol_x.view(-1)
        sol_y = sol_y.view(-1)
        
        x = torch.matmul(P_tensor, sol_x)
        y = torch.matmul(P_tensor, sol_y)

        wc_alpha = x - x_obs
        ws_alpha = y - y_obs
        alpha_obs = torch.atan2(ws_alpha * a_obs, wc_alpha * b_obs)
        
        c1_d = rho_obs * (a_obs ** 2 * torch.cos(alpha_obs) ** 2 + b_obs ** 2 * torch.sin(alpha_obs) ** 2)
        c2_d = rho_obs * (a_obs * wc_alpha * torch.cos(alpha_obs) + b_obs * ws_alpha * torch.sin(alpha_obs))
        d_temp = c2_d / c1_d
        d_obs = torch.max(d_temp, ones_tensor)
        
        res_x_obs_vec = wc_alpha - a_obs * d_obs * torch.cos(alpha_obs)
        res_y_obs_vec = ws_alpha - b_obs * d_obs * torch.sin(alpha_obs)
        
        res_eq_x_vec = torch.matmul(A_eq_tensor, sol_x) - bx_eq_tensor
        res_eq_y_vec = torch.matmul(A_eq_tensor, sol_y) - by_eq_tensor
        
        lamda_x -= rho_obs * torch.matmul(A_obs_tensor.T, res_x_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_x_vec)
        lamda_y -= rho_obs * torch.matmul(A_obs_tensor.T, res_y_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_y_vec)
    
    sol = torch.cat([sol_x, sol_y, alpha_obs.view(-1), d_obs.view(-1)])
    return sol

In [9]:
b_tensor = torch.tensor([-1.8047,  0.2048,  0.8956, -1.4917,  1.1513,  0.4862, -0.2575,  1.1148, 
                         0.8748, -0.0487, -1.0220,  0.6979], dtype=torch.double)

lamda_x_test = torch.zeros(11)
lamda_y_test = torch.zeros(11)
b_tensor.size(), lamda_x_test.size(), lamda_y_test.size()

(torch.Size([12]), torch.Size([11]), torch.Size([11]))

In [10]:
d_obs = torch.ones(num_obs, num)
alpha_obs = torch.zeros(num_obs, num)    
ones_tensor = torch.ones((num_obs, num), dtype=torch.double)
bx_eq_tensor, by_eq_tensor = torch.split(b_tensor, 6, dim=0)
cost_smoothness = weight_smoothness * torch.matmul(Pddot_tensor.T, Pddot_tensor)
cost = cost_smoothness + rho_obs * torch.matmul(A_obs_tensor.T, A_obs_tensor) + rho_eq * torch.matmul(A_eq_tensor.T, A_eq_tensor)

In [11]:
for i in range(maxiter):
    temp_x_obs = d_obs * torch.cos(alpha_obs) * a_obs
    temp_y_obs = d_obs * torch.sin(alpha_obs) * b_obs

    b_obs_x = x_obs_tensor.view(num * num_obs) + temp_x_obs.view(num * num_obs)
    b_obs_y = y_obs_tensor.view(num * num_obs) + temp_y_obs.view(num * num_obs)

    lincost_x = -lamda_x_test - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_x) - rho_eq * torch.matmul(A_eq_tensor.T, bx_eq_tensor)
    lincost_y = -lamda_y_test - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_y) - rho_eq * torch.matmul(A_eq_tensor.T, by_eq_tensor)

    lincost_x = lincost_x.view(-1, 1)
    lincost_y = lincost_y.view(-1, 1)

    sol_x, _ = torch.solve(lincost_x, -cost)
    sol_y, _ = torch.solve(lincost_y, -cost)

    sol_x = sol_x.view(-1)
    sol_y = sol_y.view(-1)

    x = torch.matmul(P_tensor, sol_x)
    y = torch.matmul(P_tensor, sol_y)

    wc_alpha = x - x_obs
    ws_alpha = y - y_obs
    alpha_obs = torch.atan2(ws_alpha * a_obs, wc_alpha * b_obs)

    c1_d = rho_obs * (a_obs ** 2 * torch.cos(alpha_obs) ** 2 + b_obs ** 2 * torch.sin(alpha_obs) ** 2)
    c2_d = rho_obs * (a_obs * wc_alpha * torch.cos(alpha_obs) + b_obs * ws_alpha * torch.sin(alpha_obs))
    d_temp = c2_d / c1_d
    d_obs = torch.max(d_temp, ones_tensor)
    # d_obs = F.relu(d_temp)

    res_x_obs_vec = wc_alpha - a_obs * d_obs * torch.cos(alpha_obs)
    res_y_obs_vec = ws_alpha - b_obs * d_obs * torch.sin(alpha_obs)

    res_eq_x_vec = torch.matmul(A_eq_tensor, sol_x) - bx_eq_tensor
    res_eq_y_vec = torch.matmul(A_eq_tensor, sol_y) - by_eq_tensor
    
    lamda_x_test -= rho_obs * torch.matmul(A_obs_tensor.T, res_x_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_x_vec)
    lamda_y_test -= rho_obs * torch.matmul(A_obs_tensor.T, res_y_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_y_vec)    

In [12]:
new_sol = torch.cat([sol_x, sol_y, alpha_obs.view(-1), d_obs.view(-1)])

In [19]:
tp = torch.tensor([  4.96621797, -19.05468109,  13.81199478,   0.0935155 ,
         0.2375418 ,   0.25648372,   0.18693714,   0.06640255,
        11.02021339, -15.01854218,   3.433404  ])

In [20]:
lamda_y_test - tp

tensor([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -5.96046448e-08,
         1.04308128e-07,  1.19209290e-07, -8.94069672e-08,  0.00000000e+00,
         9.53674316e-07,  0.00000000e+00,  2.38418579e-07])

#### Variable Size and PyTorch Debugging

In [ ]:
d_obs = torch.zeros(num_obs, num)
alpha_obs = torch.zeros(num_obs, num)
lamda_x = torch.zeros(nvar)
lamda_y = torch.zeros(nvar)
res_obs = torch.ones(maxiter)
res_eq = torch.ones(maxiter)
d_min = torch.ones(maxiter)
alpha_obs.size(), alpha_obs.size(), lamda_x.size(), lamda_y.size(), res_obs.size(), res_eq.size(), d_min.size()

In [ ]:
cost_smoothness = weight_smoothness * torch.matmul(Pddot_tensor.T, Pddot_tensor)
cost = cost_smoothness + rho_obs * torch.matmul(A_obs_tensor.T, A_obs_tensor) + rho_eq * torch.matmul(A_eq_tensor.T, A_eq_tensor)
cost_smoothness.size(), cost.size()

In [ ]:
temp_x_obs = d_obs * torch.cos(alpha_obs) * a_obs
temp_y_obs = d_obs * torch.sin(alpha_obs) * b_obs

In [ ]:
b_obs_x = x_obs_tensor.view(num * num_obs) + temp_x_obs.view(num * num_obs)
b_obs_y = y_obs_tensor.view(num * num_obs) + temp_y_obs.view(num * num_obs)
b_obs_x.size(), b_obs_y.size()

In [ ]:
bx_eq_tensor1 = params[:6, 0]
by_eq_tensor1 = params[6:, 0]
bx_eq_tensor1.size(), by_eq_tensor1.size()

In [ ]:
torch.matmul(A_obs_tensor.T, b_obs_x).size()

In [ ]:
A_eq_tensor.size(), bx_eq_tensor1.size()
# torch.matmul(A_eq_tensor.T, bx_eq_tensor1).size()

In [ ]:
lincost_x = -lamda_x - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_x) - rho_eq * torch.matmul(A_eq_tensor.T, bx_eq_tensor1)
lincost_y = -lamda_y - rho_obs * torch.matmul(A_obs_tensor.T, b_obs_y) - rho_eq * torch.matmul(A_eq_tensor.T, by_eq_tensor1)

lincost_x = lincost_x.view(-1, 1)
lincost_y = lincost_y.view(-1, 1)

In [ ]:
sol_x, _ = torch.solve(lincost_x, -cost)
sol_y, _ = torch.solve(lincost_y, -cost)

sol_x = sol_x.view(-1)
sol_y = sol_y.view(-1)

In [ ]:
x = torch.matmul(P_tensor, sol_x)
y = torch.matmul(P_tensor, sol_y)

wc_alpha = x - x_obs
ws_alpha = y - y_obs
alpha_obs = torch.atan2(ws_alpha * a_obs, wc_alpha * b_obs)

In [ ]:
c1_d = rho_obs * (a_obs ** 2 * torch.cos(alpha_obs) ** 2 + b_obs ** 2 * torch.sin(alpha_obs) ** 2)
c2_d = rho_obs * (a_obs * wc_alpha * torch.cos(alpha_obs) + b_obs * ws_alpha * torch.sin(alpha_obs))
d_temp = c2_d / c1_d
d_obs = F.relu(d_temp)

In [ ]:
res_x_obs_vec = wc_alpha - a_obs * d_obs * torch.cos(alpha_obs)
res_y_obs_vec = ws_alpha - b_obs * d_obs * torch.sin(alpha_obs)
res_x_obs_vec.size(), res_y_obs_vec.size()

In [ ]:
res_eq_x_vec = torch.matmul(A_eq_tensor, sol_x) - bx_eq_tensor1
res_eq_y_vec = torch.matmul(A_eq_tensor, sol_y) - by_eq_tensor1

In [ ]:
lamda_x -= rho_obs * torch.matmul(A_obs_tensor.T, res_x_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_x_vec)
lamda_y -= rho_obs * torch.matmul(A_obs_tensor.T, res_y_obs_vec.view(-1)) + rho_eq * torch.matmul(A_eq_tensor.T, res_eq_y_vec)